# Работа с текстами

###### библиотека NLTK - питоновская библиотека для манипуляцй с текстами

In [3]:
import nltk

##### токенизация: разбитие текста на токены
Первый этап работы с текстовой информацией - выделение слов. 

In [4]:
tokenizer = nltk.tokenize.WordPunctTokenizer()

print("WordPuckt tokenizer")
print(tokenizer.tokenize('Мама мыла раму'))
print(tokenizer.tokenize('Ростов-на-Дону'))


tokenizer = nltk.tokenize.TreebankWordTokenizer()
print("\nTreebankWord tokenizer")
print(tokenizer.tokenize('Мама мыла раму'))
print(tokenizer.tokenize('Ростов-на-Дону'))

WordPuckt tokenizer
['Мама', 'мыла', 'раму']
['Ростов', '-', 'на', '-', 'Дону']

TreebankWord tokenizer
['Мама', 'мыла', 'раму']
['Ростов-на-Дону']


##### Стемминг - приведение слов к нормальному виду


In [5]:
from nltk.stem.snowball import RussianStemmer
stemmer = RussianStemmer()
tokens = tokenizer.tokenize('Карл у Клары украл кораллы, Клара у Карла уркала кларнет.')
tokens_stem = [stemmer.stem(token.lower()) for token in tokens]
print(tokens_stem)

['карл', 'у', 'клар', 'укра', 'коралл', ',', 'клар', 'у', 'карл', 'урка', 'кларнет', '.']


#### Мешок слов
Простейшая модель для работы с текстами - 'мешок слов'. Идея заключается в том, что смысл текста можно восстановить даже не зная порядок слов, только зная их количества

In [6]:
from collections import Counter
tokens_bow = Counter(tokens_stem)
print(dict(tokens_bow))

{',': 1, 'у': 2, 'клар': 2, 'укра': 1, 'карл': 2, '.': 1, 'коралл': 1, 'урка': 1, 'кларнет': 1}


#### Корпусом в текстовом анализе называают набор документов

In [7]:
import gensim
corpora = [
    "Белый заяц бел, да цена ему пятнадцать копеек.",
    "В кулаке все пальцы равны.",
    "Глазам стыдно, а душа радуется.",
    "Где дешево, там и дорого.",
    "Дойдут и до глухого вести.",
    "И бородавка телу прибавка.",
    "Купить лишнее - продать нужное.",
    "Одним махом сто побивахом, а прочих не считахом.",
    "Смерть найдет причину.",
    "У нашего Гришки нет отрыжки."
]

corpora_tokenzied = [tokenizer.tokenize(doc.lower()) for doc in corpora]
corpora_stemmed = []

for doc in corpora_tokenzied:
    stemmed_doc = [stemmer.stem(token) for token in doc]
    corpora_stemmed.append(stemmed_doc)

for doc in corpora_stemmed:
    print(doc)

['бел', 'заяц', 'бел', ',', 'да', 'цен', 'ем', 'пятнадца', 'копеек', '.']
['в', 'кулак', 'все', 'пальц', 'равн', '.']
['глаз', 'стыдн', ',', 'а', 'душ', 'рад', '.']
['где', 'дешев', ',', 'там', 'и', 'дор', '.']
['дойдут', 'и', 'до', 'глух', 'вест', '.']
['и', 'бородавк', 'тел', 'прибавк', '.']
['куп', 'лишн', '-', 'прода', 'нужн', '.']
['одн', 'мах', 'сто', 'побивах', ',', 'а', 'проч', 'не', 'считах', '.']
['смерт', 'найдет', 'причин', '.']
['у', 'наш', 'гришк', 'нет', 'отрыжк', '.']


#### библиотека gensim содержит некоторое количество математических моделей для работы с текстами

In [8]:
import gensim

При работе с текстами не очень удобно использазовать строковые id, удобнее все слова языка пронумеровать и заменить их числами. В gensim для этого есть специальный класс Dictionary.

In [9]:
dictionary = gensim.corpora.Dictionary(corpora_stemmed)
print(dictionary.token2id['в'])

12


Теперь соберем преобразуем наш корпус в мешки слов. Для этого воспользуемся стандартным функционалом gensim

In [10]:
corpora_bow = [dictionary.doc2bow(doc) for doc in corpora_stemmed]
corpora_bow

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1)],
 [(8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)],
 [(1, 1), (8, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)],
 [(1, 1), (8, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)],
 [(8, 1), (21, 1), (24, 1), (25, 1), (26, 1), (27, 1)],
 [(8, 1), (21, 1), (28, 1), (29, 1), (30, 1)],
 [(8, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)],
 [(1, 1),
  (8, 1),
  (18, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1)],
 [(8, 1), (43, 1), (44, 1), (45, 1)],
 [(8, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1)]]

Теперь можно уже воспринимать документы не как тексты, а как вектор в N-мерном пространстве, в котором i-я координата означает количество раз, которое i-е слово встретилось в документе. 

#### частотный анализ. Посчитаем сколько раз каждое из слов встретилось в нашем корпусе


In [11]:
result = Counter()
for doc in corpora_bow:
    result += Counter(dict(doc))

for token_id, cnt in result.most_common(5):
    print(dictionary[token_id], cnt)

. 10
, 4
и 3
бел 2
а 2


##### считаем примеры текстов с которыми будем работать - база русскоязычных твитов

In [12]:
import json
twits = [json.loads(line) for line in open('twits.json')]
corpora = [twit['text'] for twit in twits]

### Задача №1. Посчитать частоты токенов в нашей базе. найти 20 самых частых. Для анализа использовать первых 5000 твитов
В процессе построить мешки слов и словарь при помощи gensim по первым 10000 

In [1]:
from tqdm import tqdm
corpora_tokenzied = []
for doc in tqdm(corpora[:10000]):
    corpora_tokenzied.append(tokenizer.tokenize(doc.lower()))
    
corpora_stemmed = []

for doc in tqdm(corpora_tokenzied):
    stemmed_doc = [stemmer.stem(token) for token in doc]
    corpora_stemmed.append(stemmed_doc)

dictionary = gensim.corpora.Dictionary(corpora_stemmed)
corpora_bow = [dictionary.doc2bow(doc) for doc in corpora_stemmed]

result = Counter()
for doc in corpora_bow:
    result += Counter(dict(doc))

for token_id, cnt in result.most_common(20):
    print(dictionary[token_id], cnt)


NameError: name 'corpora' is not defined

In [2]:
import pickle
(dictionary, corpora_bow) = pickle.load(open('twits_bow.pickle', 'rb'))

/anaconda3/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


# закон ципфа
Закон Ципфа («ранг—частота») — эмпирическая закономерность распределения частоты слов естественного языка: если все слова языка (или просто достаточно длинного текста) упорядочить по убыванию частоты их использования, то частота n-го слова в таком списке окажется приблизительно обратно пропорциональной его порядковому номеру n (так называемому рангу этого слова, см. шкала порядка).

In [ ]:
%matplotlib inline
token_freq = Counter()
for doc in tqdm(corpora_bow):
    for token,cnt in doc:
        token_freq[token] += cnt



In [ ]:
import math
from matplotlib import pyplot as plt

token_freq_list = token_freq.most_common(100)
rank = list(range(len(token_freq_list)))
freq = []
for i in rank:
    freq.append(1/token_freq_list[i][1])
    
plt.plot(rank, freq)

Самые частые слова являются наименее информативными. Хочется придумать какую-то модель, которая будет давать больший вес более редким словам и меньший вес более частым. 

# tf-idf

В модели tf-idf вместо просто количества слова используем количество домноженное на коэффицент idf, который уменьшает вес частых слов

![tf](https://wikimedia.org/api/rest_v1/media/math/render/svg/92a19022b85d3796b7e6237ea6829cb550ef17ff)
![idf](https://wikimedia.org/api/rest_v1/media/math/render/svg/1c1f3347300bd19654bedfaef73861cf75ac5e65)

doc[word] = tf[doc][word] * idf[word]


в gensim есть специальный модуль для работы с tfidf

In [ ]:
tfidf = gensim.models.TfidfModel(corpora_bow)
corpora_tfidf = tfidf[corpora_bow]

## sentiment analysis

Теперь у нас есть хорошее представление наших твитов. Попробуем обучить логистическую регрессию, которая научится отличать негативные твиты от позитивных

In [1]:
target = [twit['sentiment'] for twit in twits]

NameError: name 'twits' is not defined

In [32]:
from scipy.sparse import csc_matrix
try:
    (data, col, row) = pickle.load(open('matrix.pickle', 'rb'))
except:
    data = []
    col = []
    row = []
    for i in tqdm(range(len(corpora_tfidf))):
        for j in range(len(corpora_tfidf[i])):
            data.append(corpora_tfidf[i][j][1])
            col.append(corpora_tfidf[i][j][0])
            row.append(i)
    pickle.dump((data, col, row), open('matrix.pickle', 'wb'))

In [ ]:
matrix = csc_matrix((data, (row, col)), shape=(len(corpora_tfidf), len(dictionary)))
from sklearn.linear_model import LogisticRegression

In [ ]:
classifier = LogisticRegression()
classifier.fit(matrix, target)


In [31]:
coef_weight = list(enumerate(classifier.coef_[0]))
coef_weight.sort(key=lambda x: x[1])
for word in coef_weight[:50]:
    print(dictionary[word[0]])

NameError: name 'classifier' is not defined

In [29]:
def classify(text):
    tokens = [stemmer.stem(token) for token in tokenizer.tokenize(text.lower())]
    bow = dictionary.doc2bow(tokens)
    tfidf_vec = tfidf[bow]
    data = []
    col = []
    row = []
    for token in tfidf_vec:
        data.append(token[1])
        col.append(token[0])
        row.append(0)
    matrix_1 = csc_matrix((data, (row, col)), shape=(1, len(dictionary)))
    return(classifier.predict_proba(matrix_1)[0][1])

In [47]:
print(classify("наконец-то 0.7"))

0.628812956025


In [45]:
print(classify("хочу умереть, ненавижу всех"))

0.1551585591


# Problem 2
Разбить выборку на train и test и померить качество
Воспользоваться методами sklearn.cross_validation.train_test_split для разбивки и sclearn.metrics.roc_auc_score для измерения качества

In [48]:
from sklearn.cross_validation import train_test_split
data_train, data_test, target_train, target_test = train_test_split(matrix, target)
clsf = LogisticRegression()
clsf.fit(data_train, target_train)
result = clsf.predict_proba(data_test)


from sklearn.metrics import roc_auc_score
roc_auc_score(target_test, result[:, 1])

0.99990025221766865

# topic modeling

Попробуем выяснить какие темы волнуют пользователей твиттера. Для этого воспользуемся моделью latent dirichlet allocation

In [49]:
try:
    lda = pickle.load(open("lda_model.pickle", 'rb'))
except:
    lda = gensim.models.ldamulticore.LdaMulticore(corpus=corpora_tfidf, id2word=dictionary)
    pickle.dump(lda, open("lda_model.pickle", 'wb'))

In [51]:
for topic in lda.print_topics(30):
    print(topic)

(47, '0.009*( + 0.007*) + 0.007*зуб + 0.006*, + 0.006*замерзл + 0.005*! + 0.005*в + 0.004*и + 0.004*я + 0.004*не')
(62, '0.010*английск + 0.010*( + 0.007*черт + 0.006*заболет + 0.006*) + 0.006*, + 0.005*одиночеств + 0.005*болел + 0.005*: + 0.004*не')
(98, '0.010*учеб + 0.010*( + 0.009*ретв + 0.007*) + 0.006*проеба + 0.005*заснут + 0.005*, + 0.005*останов + 0.005*похож + 0.005*@')
(60, '0.014*тяжел + 0.008*( + 0.006*глуп + 0.006*дни + 0.006*) + 0.005*, + 0.005*! + 0.005*я + 0.005*не + 0.005*в')
(92, '0.014*( + 0.009*зачет + 0.007*! + 0.006*, + 0.006*9 + 0.006*встрет + 0.006*не + 0.006*) + 0.006*числ + 0.005*я')
(2, '0.013*( + 0.010*чет + 0.010*шерлок + 0.008*отвеча + 0.007*экзамен + 0.006*смогл + 0.006*уста + 0.006*врач + 0.005*не + 0.005*учител')
(36, '0.008*пизд + 0.007*где-т + 0.007*) + 0.006*приложен + 0.005*пролетел + 0.005*( + 0.005*: + 0.005*, + 0.005*@ + 0.005*лер')
(15, '0.011*фу + 0.008*( + 0.007*увидет + 0.007*аааа + 0.007*! + 0.006*трен + 0.006*) + 0.005*вспомина + 0.005*, +